# access -- ITS_LIVE data access from object storage

> These are tools for accessing ITS_LIVE velocity data from object storage (s3 buckets) and preliminary organization of dataset 

In [ ]:
#| default_exp access

In [ ]:
#| hide
from nbdev.showdoc import *
import nbdev 
from nbdev import nbdev_export

In [ ]:
#| export 

import os
import json


import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import Polygon

import numpy as np
import xarray as xr
import rioxarray as rxr

import geoviews as gv
import geoviews.feature as gf
import hvplot.pandas

import s3fs

In [ ]:
#| hide

itslive_catalog = gpd.read_file('https://its-live-data.s3.amazonaws.com/datacubes/catalog_v02.json')


In [ ]:
#| export
def read_in_s3(http_url, chunks='auto'):
    ''' does some string formatting from zarr url and returns xarray dataset
    '''

    s3_url = http_url.replace('http','s3')
    s3_url = s3_url.replace('.s3.amazonaws.com','')

    datacube = xr.open_dataset(s3_url, engine='zarr',
                               storage_options={'anon':True},
                               chunks=chunks)
    return datacube
    
def find_granule_by_zone(zone):
    '''returns every url from itslive catalog that corresponds to a granule falling in specified UTM zone
    '''

    catalog = gpd.read_file('https://its-live-data.s3.amazonaws.com/datacubes/catalog_v02.json')

    zone_cat = catalog.loc[catalog['epsg'] == zone]

    return list(zone_cat['zarr_url'].values)

def find_granule_by_point(input_point, label='single_point'):
    '''returns url for the granule (zarr datacube) containing a specified point. point must be passed in epsg:4326
    '''
    catalog = gpd.read_file('https://its-live-data.s3.amazonaws.com/datacubes/catalog_v02.json')

    #make shapely point of input point
    p = gpd.GeoSeries([Point(input_point[0], input_point[1])],crs='EPSG:4326')
    #make gdf of point
    gdf = gdf = gpd.GeoDataFrame({'label': f'{label}', 
                                  'geometry':p})

    #find row of granule 
    granule = catalog.sjoin(gdf, how='inner')

    url = granule['zarr_url'].values[0]
    return url



In [ ]:
#| hide
zone_urls = find_granule_by_zone(32646)

In [ ]:
#| hide
point = [95.180191, 30.645973]

In [ ]:
#| hide
point_url = find_granule_by_point(point, 'test_label')
point_url

'http://its-live-data.s3.amazonaws.com/datacubes/v02/N30E090/ITS_LIVE_vel_EPSG32646_G0120_X750000_Y3350000.zarr'

In [ ]:
#| hide
da = read_in_s3(point_url)

In [ ]:
#| hide
da

<xarray.Dataset>
Dimensions:                    (mid_date: 6554, y: 833, x: 833)
Coordinates:
  * mid_date                   (mid_date) datetime64[ns] 2021-05-13T04:30:01....
  * x                          (x) float64 7.001e+05 7.003e+05 ... 8e+05
  * y                          (y) float64 3.4e+06 3.4e+06 ... 3.3e+06 3.3e+06
Data variables: (12/54)
    acquisition_date_img1      (mid_date) datetime64[ns] dask.array<chunksize=(6554,), meta=np.ndarray>
    acquisition_date_img2      (mid_date) datetime64[ns] dask.array<chunksize=(6554,), meta=np.ndarray>
    autoRIFT_software_version  (mid_date) <U5 dask.array<chunksize=(6554,), meta=np.ndarray>
    chip_size_height           (mid_date, y, x) float32 dask.array<chunksize=(6554, 70, 70), meta=np.ndarray>
    chip_size_width            (mid_date, y, x) float32 dask.array<chunksize=(6554, 70, 70), meta=np.ndarray>
    date_center                (mid_date) datetime64[ns] dask.array<chunksize=(6554,), meta=np.ndarray>
    ...                         ...
    vy_error_mask              (mid_date) float64 dask.array<chunksize=(6554,), meta=np.ndarray>
    vy_error_modeled           (mid_date) float64 dask.array<chunksize=(6554,), meta=np.ndarray>
    vy_error_slow              (mid_date) float64 dask.array<chunksize=(6554,), meta=np.ndarray>
    vy_stable_shift            (mid_date) float64 dask.array<chunksize=(6554,), meta=np.ndarray>
    vy_stable_shift_mask       (mid_date) float64 dask.array<chunksize=(6554,), meta=np.ndarray>
    vy_stable_shift_slow       (mid_date) float64 dask.array<chunksize=(6554,), meta=np.ndarray>
Attributes: (12/18)
    GDAL_AREA_OR_POINT:         Area
    author:                     ITS_LIVE, a NASA MEaSUREs project (its-live.j...
    autoRIFT_parameter_file:    http://its-live-data.s3.amazonaws.com/autorif...
    datacube_software_version:  1.0
    date_created:               09-Jun-2022 09:28:39
    date_updated:               09-Jun-2022 09:28:39
    ...                         ...
    s3:                         s3://its-live-data/datacubes/v02/N30E090/ITS_...
    skipped_granules:           s3://its-live-data/datacubes/v02/N30E090/ITS_...
    time_standard_img1:         UTC
    time_standard_img2:         UTC
    title:                      ITS_LIVE datacube of image_pair velocities
    url:                        https://its-live-data.s3.amazonaws.com/datacu...

Now, want to easily/quickly see the location of the granules. Use `get_bbox()` to get a Geopandas.GeoDataFrame object representing the bounding box of the velocity object.

In [ ]:
#| export
def get_bbox(input_xr, ref='latlon'):
    '''returns geopandas geodataframe of bounding box of xarray object. 
    '''

    epsg=input_xr.projection

    xmin = input_xr.coords['x'].data.min()
    xmax = input_xr.coords['x'].data.max()

    ymin = input_xr.coords['y'].data.min()
    ymax = input_xr.coords['y'].data.max()

    pts_ls = [(xmin, ymin), (xmax, ymin),(xmax, ymax), (xmin, ymax), (xmin, ymin)]

    crs = f'epsg:{epsg}'

    polygon_geom = Polygon(pts_ls)
    polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom]) 

    #polygon = polygon.to_crs('epsg:4326')

    return polygon
    

In [ ]:
bbox = get_bbox(da)

The returned bbox (`geopandas.GeoDataFrame`) object will have the same CRS as the da (`xarray.Dataset`) object. Re-project data to WGS84 to view interactively:  

In [ ]:
poly = bbox.to_crs('epsg:4326').hvplot(geo=True, legend=True,alpha=0.3, tiles='ESRI', color='red')

In [ ]:
poly

:Overlay
   .Tiles.I    :Tiles   [x,y]
   .Polygons.I :Polygons   [Longitude,Latitude]